In [52]:
import popy as popy
from popy.data_fakers.soep import soep_faker
from popy.pop_maker import PopMaker

from popy.location import WeightedLocation

import pandas as pd
import random
import matplotlib.pyplot as plt

import datetime as dt

In [53]:
class Agent(popy.Agent):
    def __init__(self, model, *args, **kwargs):
        super().__init__(model, *args, **kwargs)

        self.infection_status = "susceptible"
        self.days_since_infection = 0
    

    def infect(self):
        if self.infection_status in ["infectious", "symptomatic"]:
            for location in self.locations:
                for agent_v in location.neighbors(self):
                    if agent_v.infection_status == "susceptible":

                        contact_weight = location.edge_weight_simultan(self, agent_v, denominator=24)
                        # -> eine Agenten-Methode schreiben, die das Gewicht zu einem anderen Agenten über alle Locations wiedergibt
                        
                        infection_probability = self.model.p.infection_weight * contact_weight
                        
                        if random.random() < infection_probability:
                            agent_v.infection_status = "exposed"
    

    def update_infection_status(self):
        if self.infection_status in ["exposed", "infectious", "symptomatic"]:
            self.days_since_infection += 1

            if 2 <= self.days_since_infection <= 5:
                self.infection_status = "infectious"
            
            elif 6 <= self.days_since_infection <= 10:
                self.infection_status = "symptomatic"
            
            elif self.days_since_infection >= 11:
                self.infection_status = "recovered"

In [54]:
class Home(WeightedLocation):
    def groupby(self, agent):
        return agent.hid
    
    def weight(self, agent):
        return 8


class Work(WeightedLocation):
    def setup(self):
        self.size = 10
    
    def join(self, agent):
        return agent.work_hours_day > 0

    def weight(self, agent):
        return agent.work_hours_day


class School(WeightedLocation):
    def setup(self):
        self.size = 25
    
    def join(self, agent) -> bool:
        return 6 <= agent.age <= 18
    
    def weight(self, agent):
        weight = 0
        
        # workday?
        if agent.model.date.weekday() < 5:
            # not ill?
            if agent.infection_status != "symptomatic" or agent.infection_status == "symptomatic" and random.random() < 0.5:
                # no holidays?
                if random.random() < self.model.get_timetable_entry()["school"]:
                    weight = 6

        return weight


class Friendship(WeightedLocation):
    def setup(self):
        self.size = 2
    
    def groupby(self, agent):
        return (abs(agent.age + random.randint(-5,5)), agent.gender)
    
    def weight(self, agent):
        return 2 if agent.model.date.weekday() >= 5 else 0

In [55]:
class Model(popy.Model):
    
    def setup(self):
        self.date = dt.date(2022, 1, 1)
        df_soep = soep_faker.soep(size=self.p.n_agents)
        self.pop_maker = PopMaker(model=self)
        self.agents = self.pop_maker.create_agents(df=df_soep, agent_class=Agent)
        self.locations = self.pop_maker.create_locations(agents=self.agents, location_classes=[Home, School, Work, Friendship])
        
        self.infection_list = []

        for agent in random.choices(self.agents, k=10):
            agent.infection_status = "exposed"
        
        self.timetable = {
            dt.date(2020, 1, 1): {
                "school": 1,
            },
            dt.date(2020, 2, 1): {
                "school": 0,
            },
            dt.date(2020, 2, 15): {
                "school": 1,
            }
        }
        

    def step(self):
        self.agents.update_infection_status()
        self.agents.infect()
        
    def update(self):
        self.record("cumulative_infections", sum([1 for agent in self.agents if agent.infection_status != "susceptible"]))
        self.date = self.date + dt.timedelta(days=1)
    
    def get_timetable_entry(self):
        for date_key in self.timetable:
            if date_key <= self.date:
                entry = self.timetable[date_key]
        return entry

In [56]:
parameters = {
    "steps": 50,
    "n_agents": 1000,
    "infection_weight": 0.1,
}
model = Model(parameters=parameters)

results = model.run()

plt.plot(results.variables.Model.cumulative_infections)

Completed: 26 steps

In [ ]:
model.pop_maker.eval_affiliations()



______________________________________
Number of agents per location
______________________________________

               n_agents                                                   
                  count       mean       std   min   25%   50%   75%   max
location_class                                                            
Friendship        505.0   1.922772  0.570415   0.0   2.0   2.0   2.0   3.0
Home              321.0   3.115265  1.547917   1.0   2.0   3.0   4.0  10.0
School              3.0  28.333333  1.527525  27.0  27.5  28.0  29.0  30.0
Work               43.0  10.023256  0.152499  10.0  10.0  10.0  10.0  11.0


______________________________________
Number of affiliated locations per agent
______________________________________

count    1000.000000
mean        2.487000
std         1.043513
min         1.000000
25%         2.000000
50%         2.000000
75%         3.000000
max         6.000000
Name: n_affiliated_locations, dtype: float64
